In [37]:
from torch.utils.data import DataLoader, random_split
from torch.optim import Adam, RMSprop
from data.preprocessing import *
from data.data_utils import *
from models.macro_architectures import *
from models.mult2multatt import * # cambiar respecto al modelo
from models.sing2multatt import *
from utils import *

In [38]:
device = get_default_device()

In [66]:
#DATA PROCESSING
start_time = '20210101'
end_time = '20230802'
scrap_date = interval_time(start_time, end_time)
months = list(set([day[:6] for day in scrap_date]))
import_Dst(months)
l1_sample, l2_sample, dst, kp = automated_preprocessing(scrap_date, sep = True)
l1_sample_hour = (l1_sample[0].resample('60min').mean(), l1_sample[1].resample('60min').mean())

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x00000236BC24E950>
Traceback (most recent call last):
  File "C:\Users\jorge\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torch\utils\data\dataloader.py", line 1479, in __del__
    self._shutdown_workers()
  File "C:\Users\jorge\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torch\utils\data\dataloader.py", line 1437, in _shutdown_workers
    if self._persistent_workers or self._workers_status[worker_id]:
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_workers_status'


In [67]:
sequence_length_hour = 5  #hour
sequence_length_minute = 300 #minute
pred_length = 5 #hours
dict_values = ['dst_kyoto', 'kp_gfz']

In [68]:
hour_kp_dataset = KpData(l1_sample_hour, kp, 5, 3, hour=True, sep = True)

In [70]:
hour_kp_dataset[0]

(tensor([[ 0.6947, -0.2811,  0.8946,  0.6947, -0.4846,  0.9092, -0.7058, -0.8674,
          -0.6802],
         [ 0.7772, -0.1330,  0.6567,  0.7772, -0.2686,  0.7084, -0.7872, -1.0368,
          -0.6555],
         [ 0.8702, -0.2795,  0.8280,  0.8702, -0.4158,  0.8505, -0.8776, -0.9053,
          -0.5869],
         [ 0.8070, -0.3426,  0.7007,  0.8070, -0.4240,  0.7170, -0.8156, -0.9876,
          -0.6777],
         [ 0.8213, -0.4504,  1.0703,  0.8213, -0.5345,  1.0802, -0.8208, -0.9814,
          -0.6950]]),
 tensor([[-1.0359, -0.1740, -0.4173,  0.5256,  1.3772,  0.7109, -0.1740, -0.5139,
           0.3680,  0.8780,  0.7154],
         [-1.0666,  0.2727, -0.1642,  0.5733,  1.4863,  1.0919,  0.2727, -0.2615,
           0.4990,  1.2113,  1.0995],
         [-1.1024,  0.5748, -0.1160,  0.2709,  0.6985,  1.1058,  0.5748, -0.1546,
           0.2422,  0.5814,  1.1135],
         [-1.0958,  0.3035, -0.3011,  0.2235,  0.6140,  1.1074,  0.3035, -0.3341,
           0.1703,  0.4194,  1.1151],
        

In [71]:
#Test:5% training: 95%

test_size = round(0.05*len(hour_kp_dataset))

train_hour_kp, test_hour_kp = random_split(hour_kp_dataset , [len(hour_kp_dataset) - test_size, test_size])

batch_size = 32  #Change based on GPU capacity

train_hour_kp_dl = DataLoader(train_hour_kp, batch_size, shuffle=True, num_workers=4, pin_memory=True)
train_hour_kp_dl = DeviceDataLoader(train_hour_kp_dl, device)
test_hour_kp_dl = DataLoader(test_hour_kp, batch_size*2, num_workers=4, pin_memory=True)
test_hour_kp_dl = DeviceDataLoader(test_hour_kp_dl, device)

In [76]:
encoder_fc = EncoderMultiheadAttentionGRU(9, 10, 3, (5,5))
encoder_mg = EncoderMultiheadAttentionGRU(11, 10, 11, (5,5))
model  = MultiHeaded2MultiheadAttentionGRU(encoder_fc, encoder_mg, [3,1], (5,5), 1)

In [77]:
##hyperparameters
epochs = 20
max_lr = 1e-2
weigth_decay = 1e-6
grad_clip = 1e-1
opt_func = Adam
#opt_func = RMSprop

In [78]:
model.fit(epochs, max_lr,train_hour_kp_dl, test_hour_kp_dl, weigth_decay, grad_clip, opt_func)

Epoch [0]:
	last_lr: 0.00104
	train_loss: 0.7306
	val_loss: 0.3733


KeyboardInterrupt: 